Import Library

In [207]:
import pandas as pd
import torch
import accelerate
import chardet
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from typing import Optional
from torch.utils.data import Dataset as TorchDataset
from typing import Optional, Dict, List
from datasets import Dataset as HFDataset

 Initialize the dataset with texts and labels.
        
        Args:
            texts: List of problem texts
            labels: List of corresponding answers
            tokenizer: BERT tokenizer instance
            max_length: Maximum sequence length for padding/truncation

In [ ]:
class ProbabilityModels(TorchDataset) : 
    def __init__(self, texts : List[str], labels : List[int], tokenizer: BertTokenizer, max_length: int = 128):
        self.encodings = tokenizer(
            texts, 
            truncation=True, 
            padding=True, 
            max_length=max_length, 
            return_tensors="pt"
        )
        self.labels = torch.tensor(labels, dtype=torch.float)

Custom Modeling for probability Problem

In [ ]:
def __len__(self) -> int:
    return len(self.labels)
    
def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
    item = {
        key: val[idx] for key, val in self.encodings.items()
    }
    item["labels"] = self.labels[idx]
    return item
    

Load and prepare data for training.
    
    Args:
        file_path: Path to the CSV file
        test_size: Proportion of data to use for validation
        random_state: Random seed for reproducibility
    
    Returns:
        Tuple of (train_dataset, eval_dataset)

In [ ]:
def load_and_prepare_data(
    file_path: str,
    test_size: float = 0.2,
    random_state: int = 42

) -> tuple[ProbabilityModels, ProbabilityModels]:

In [ ]:
//

In [209]:
data = pd.read_csv('stat_problem.csv', sep=',', encoding='cp1252')
#print(data.head()) Head hanya menampilkan 5 Index awal

In [210]:
data_clean = data.dropna(subset=['problem', 'answer'])

problem = data_clean['problem'].tolist()
res = data_clean['answer'].tolist()


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encoded = tokenizer(problem, padding=True, truncation=True, max_length=128, return_tensors='pt')

# tokenisasi Dataset
def tokenize_dataset(example) : 
    return tokenizer(
        example['problem'],
        truncation=True,
        padding=True,
    )


#split dataset menjadi data eval dan data train

train_text, val_text, train_label, val_label = train_test_split(
    problem,
    res,
    test_size=0.2 # 20% evaluasi
)


Tokenisasi DataSplit trainText dan evalText

In [211]:
train_encoding = tokenizer(train_text, truncation=True, padding=True, max_length=128, return_tensors="pt")
eval_encoding = tokenizer(val_text, truncation=True, padding=True, max_length=128, return_tensors="pt")

Buat dataset Torch Agar bisa di proses model BERT

Load and prepare data for training.

    Args:
        file_path: Path to the CSV file
        test_size: Proportion of data to use for validation
        random_state: Random seed for reproducibility
    
    Returns:
        Tuple of (train_dataset, eval_dataset)

In [213]:
train_dataset = PeluangDataset(encodings=encoded, labels=res)
eval_dataset = PeluangDataset(encodings=eval_encoding, labels=val_label)

In [214]:
# model pre Trained
model_pretrained = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [215]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cpu


Create Function Model Tunning

In [ ]:
def setup_training(
    model,
    tokenizer,
    train_dataset: Dataset,
    eval_dataset: Optional[Dataset] = None, 
    output_dir: str = './output',
    num_epochs: int = 3,
    batch_size: int = 8,
    learning_rate: float = 5e-5,
    weight_decay: float = 0.1
):  
    pass

Validate Input

In [ ]:
if not isinstance(train_dataset, Dataset):
        raise ValueError("train_dataset must be a Hugging Face Dataset")
if eval_dataset is not None and not isinstance(eval_dataset, Dataset):
        raise ValueError("eval_dataset must be a Hugging Face Dataset")

In [ ]:
# set fine tuning models
training_args = TrainingArguments(
    output_dir='./output',
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.1,
    logging_dir='./logs',
    logging_steps=10,
    no_cuda=device== "cpu"
)

trainer = Trainer(
    model=model_pretrained,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

trainer.train()